In [ ]:
# pip intall duckdb==0.7.0 rich==2.14.0

In [ ]:
from IPython.display import display
import altair as alt

def VegaLite(spec):
    bundle = {}
    bundle['application/vnd.vegalite.v4+json'] = spec
    display(bundle, raw=True)

# Récuperer les données

Pour changer la classification du produit utilisée pour faire les agrégations, changer la valeur de `PRODUCT_CLASSIFICATION` pour le nom d'une autre colonne.

In [ ]:
import duckdb

# Le dataset
CSV = '../../data/toflit18_all_flows.csv'

PRODUCT_CLASSIFICATION = 'product_revolutionempire'

# Lire le csv et l'envoyer comme une relation (table)
csv_data = duckdb.sql(f"""
SELECT  customs_region,
        customs_office,
        CAST(value AS FLOAT) as value,
        year,
        export_import,
        best_guess_region_prodxpart,
        {PRODUCT_CLASSIFICATION},
FROM read_csv_auto('{CSV}', ALL_VARCHAR=TRUE)
""")

# Fonction pour calculer l'indice

In [ ]:
def carre_du_marche(year, bureau_OR_direction, nom):

        relation = duckdb.sql(f"""
        FROM csv_data
        WHERE value IS NOT NULL
        AND export_import = 'Exports'
        AND best_guess_region_prodxpart = '1'
        AND year = '{year}'
        AND {bureau_OR_direction} = '{nom}'
        """)

        all_product_value = relation.sum('value').fetchone()[0]
        if not all_product_value:
                return None

        aggreg = duckdb.sql(f"""
        SELECT {PRODUCT_CLASSIFICATION}, SUM(value) AS sum_value
        FROM relation
        GROUP BY {PRODUCT_CLASSIFICATION}
        """)

        carre = duckdb.sql(f"""
        SELECT  {PRODUCT_CLASSIFICATION},
                sum_value, {all_product_value} AS total,
                power(sum_value/{all_product_value}, 2) AS carre_part_marche
        FROM aggreg
        """)

        return carre.sum('carre_part_marche').fetchone()[0]

# Résultats

## A. Indices par direction de ferme

Récupérer toutes les années enregistrées dans le dataset.

In [ ]:
year_select = duckdb.query('SELECT CAST(year AS INTEGER) FROM csv_data')
years = [tuple[0] for tuple in year_select.distinct().fetchall()]

Boucler sur les directions ciblées et lancer la fonction de calcul pour toute année. 

In [ ]:
# Attention: Le calcul de l'indice pour toutes les années prend environ entre 5 et 6 minutes par direction
from rich.progress import track

carre_du_marche_directions = []

directions = ['Bordeaux', 'Marseille', 'Rouen', 'Nantes', 'La Rochelle', 'Bayonne']

for direction in directions:
    print(direction)

    for year in track(years, description='Calculating indices...'):

        indice = carre_du_marche(
                year=year,
                bureau_OR_direction='customs_region',
                nom=direction)

        carre_du_marche_directions.append({
            'direction': direction,
            'year': year,
            'indice' : indice,
        })

Sauveguarder les résultats dans un fichier csv.

In [ ]:
import csv
with open('part_du_marche_des_directions.csv', 'w') as of:
    fieldnames = ['direction', 'year', 'indice']
    writer = csv.DictWriter(of, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(carre_du_marche_directions)

Ordonner et formatter les résultats en préparation pour la visualisation.

In [ ]:
relation = duckdb.read_csv('part_du_marche_des_directions.csv', header=True).order('year').fetchdf()

data = relation.to_dict('records')

Visualiser les résultats.

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Diversification des produits du commerce des principaux ports français",
    "width": 800,
    "height": 300,
    "mark": {
        "tooltip": {
         "content": "data"
        },
        "type": "line",
        "point": True
    },
    "data": {
        "values": data,
    },
    "encoding": {
        "x": {
            "field": "year",
            "type": "quantitative",
            "title": "Année",
            "axis": {
                "orient": "bottom",
            },
            "scale": {"domain": [1715, 1795]}
        },
        "y": {
            "field": "indice",
            "type": "quantitative",
            "title": "Indice Herfindhal",
        },
        "color": {
            "field": "direction",
            "type": "nominal",
            "title": "direction"
        }
    }
})

## B. Indices par bureau de ferme

Boucler sur les bureaux ciblés et lancer la fonction de calcul pour l'année 1789, qui est la seule qui ont les données nécessaires.

In [ ]:
from rich.progress import track

carre_du_marche_bureaux = []

bureaux = ['Bordeaux', 'Marseille', 'Rouen', 'Nantes', 'La Rochelle', 'Bayonne', 'Saint-Malo', 'Le Havre']
year = 1789

for bureau in track(bureaux):
    indice = carre_du_marche(
                year=year,
                bureau_OR_direction='customs_office',
                nom=bureau)
    carre_du_marche_bureaux.append({
            'bureau': bureau,
            'year': year,
            'indice' : indice,
        })

Sauvegarder les résultats dans un fichier csv.

In [ ]:
import csv
with open('part_du_marche_des_bureaux.csv', 'w') as of:
    fieldnames = ['bureau', 'year', 'indice']
    writer = csv.DictWriter(of, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(carre_du_marche_bureaux)

Ordonner et formatter les données en préparation pour la visualisation.

In [ ]:
relation = duckdb.read_csv('part_du_marche_des_bureaux.csv', header=True).order('bureau').fetchdf()

data = relation.to_dict('records')

Visualiser les résultats.

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Diversification des produits du commerce des principaux bureaux de ferme en 1789",
    "width": 800,
    "height": 300,
    "mark": {
        "tooltip": {
         "content": "data"
        },
        "type": "bar",
    },
    "data": {
        "values": data
    },
    "encoding": {
        "x": {
            "field": "bureau",
            "type": "nominal", 
            "title": "Bureau",
            "axis": {
                "orient": "bottom", 
            },
        },
        "y": {
            "field": "indice",
            "type": "quantitative",
            "title": "Indice Herfindhal"
        },
    },
})

## C. Toute la France en moyenne

Pour calculer l'indice pour toute la France, au lieu d'une direction, reprendre tout `customs_region` où la valeur n'est pas `NULL`.

In [ ]:
relation = duckdb.sql(f"""
    FROM csv_data
    WHERE customs_region IS NOT NULL
    AND year = 1789
    """)

all_product_value = relation.sum('value').fetchone()[0]

aggreg = duckdb.sql(f"""
SELECT product_revolutionempire, SUM(value) AS sum_value
FROM relation
GROUP BY product_revolutionempire
""")

carre = duckdb.sql(f"""
SELECT  product_revolutionempire,
        sum_value, {all_product_value} AS total,
        power(sum_value/{all_product_value}, 2) AS carre_part_marche
FROM aggreg
""")

carre.sum('carre_part_marche').fetchone()[0]